## Assignment 3 - Competitive Assignment
An explanation this assignment could be found in the .pdf explanation document

### Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [658]:
name = "daniel barnur"
id= 313260770
# imports for reading and writing (input & output) files:
import pandas as pd
import os
import numpy as np

In [659]:
### add whatever imports you need
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
# function to split the data for cross-validation
from sklearn.model_selection import train_test_split
# function for encoding categories
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn import metrics
# the Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import hebrew_tokenizer as ht
from sklearn.metrics import f1_score

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [662]:
train_filename = '.' + os.sep + 'input' + os.sep + 'annotated_corpus_for_train_dup_female.xlsx'
test_filename  = '.' + os.sep + 'input' + os.sep + 'corpus_for_test.xlsx'
df_train = pd.read_excel(train_filename, 'corpus', index_col=None, na_values=['NA'])
df_test  = pd.read_excel(test_filename,  'corpus', index_col=None, na_values=['NA'])

In [663]:
df_train.head(8)

,story,gender
0,"בוקר אחד קמתי סהרורי יצאתי מהמיטה קצת מטושטש ,...",m
1,לחבר שלי היה יום הולדת וחיפשנו מה אפשר לעשות ל...,m
2,"השנה האחרונה הייתה שנת קורונה, שנה לא פשוטה בק...",m
3,"לפני כחצי שנה עברתי לגור בצפון עם בת זוגתי, עב...",m
4,"יום חמישי רגיל, תמיד מתחיל לעבור טיפה מאוחר יו...",m
5,בשנה האחרונה חוויתי את מגפת הקורונה שהכריח את ...,m
6,בסמסטר קודם אני וכמה חברים ללימודים קבענו להיפ...,m
7,בשנה האחרונה למרות שלא היו יותר מידיי דברים לע...,f


In [664]:
df_test.head(3)

,test_example_id,story
0,0,כחלק ממסגרת ההתנדבות שלי במגלה אני הולך לפעמיי...
1,1,"לפני שנה החלטתי שאני רוצה להיות טייס, התחלתי ל..."
2,2,"בתקופת הקורונה של תחילת החיסונים נגד קורונה, א..."


### Your implementation:
Write your code solution in the following code-cells

In [665]:
# YOUR CODE HERE
#function to countVectorizer to "tokenizer"
def my_tokenizer(hebrew_text):
    # tokenize returns a generator! 
    
    hebrew_text2= hebrew_text. 
    return ht.tokenize(hebrew_text)

In [666]:
# YOUR CODE HERE
# creates a list of stop words from stop_word_list_heb_example.txt
my_file = open("./stop_word_list_heb_example.txt", encoding="utf8")
content = my_file.read()
content_stop= content. split("\n");
my_file. close()

In [667]:
# setting our TRAIN set to two groups
# train_set=df_train.tail(407-72)
# validation_set=df_train.head(72)
train_set ,validation_set = train_test_split(df_train,test_size=0.2) 


In [696]:
# creating a pipline with countVector includes stop words max features and tokens on our train set
# we normalize (l1) it and then use naive bayes
# stop_words= content_stop,
text_clf_nb = Pipeline([
    ('vect', CountVectorizer(stop_words=content_stop,max_features=50000, tokenizer=my_tokenizer)),
    ('norm', preprocessing.Normalizer(norm='l1')),
    ('clf', SVC()),
])

In [697]:
text_clf_nb.fit(train_set.story,train_set.gender)

Pipeline(steps=[('vect',
                 CountVectorizer(max_features=50000,
                                 stop_words=['לא', 'את', 'זה', 'של', 'על',
                                             'אני', 'מה', 'עם', 'הוא', 'כל',
                                             'אבל', 'גם', 'לי', 'יש', 'אם',
                                             'אתה', 'רק', 'או', 'היה', 'אין',
                                             'יותר', 'אז', 'היא', 'כי', 'לך',
                                             'טוב', 'הם', 'מי', 'כמו', 'שלא', ...],
                                 tokenizer=<function my_tokenizer at 0x0000028BD1F1F4C0>)),
                ('norm', Normalizer(norm='l1')), ('clf', SVC())])

In [698]:
# using on validation set to test our model
predict = text_clf_nb.predict(validation_set.story)

In [699]:
np.mean(predict == validation_set.gender)

0.8260869565217391

In [700]:
# built in classification report with f1_score for f and m and also macro avg for our Average_F1 of both
print(metrics.classification_report(validation_set.gender, predict))

              precision    recall  f1-score   support

           f       0.89      0.53      0.67        30
           m       0.81      0.97      0.88        62

    accuracy                           0.83        92
   macro avg       0.85      0.75      0.77        92
weighted avg       0.84      0.83      0.81        92



In [701]:
# another evaluation of the f1_score 

In [702]:
# function to calculate the Confusion Matrix 
# the function is fitted to the gender we need to check f | m
def perf_measure(y_actual, y_hat, g1, g2):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==g1:
            TP += 1
        if y_hat[i]==g1 and y_actual[i]!=y_hat[i]:
            FP += 1
        if y_actual[i]==y_hat[i]==g2:
            TN += 1
        if y_hat[i]==g2 and y_actual[i]!=y_hat[i]:
            FN += 1
 
    return(TP, FP, TN, FN)

In [703]:
# recall tp / (tp + fn) 
# precision tp / (tp + fp)
# Evaluation of F1_Score by gender
def evaluateF1(y_test, y_predict, gender):
    if gender=='m':
            tp, fp, TN, fn  = perf_measure(y_test, y_predict,'m','f')
    else:
        tp, fp, TN, fn  = perf_measure(y_test, y_predict,'f','m')
    precision = tp / (tp + fp)
    recall= tp / (tp + fn) 
    f1=2*(precision * recall)/(precision + recall)
    return f1

In [704]:
# evaluation f1 score
# criteria of the function @function perf_measure is to be a list
toList=list(validation_set.gender)
f1_male = evaluateF1(toList,predict, 'm')
f1_female = evaluateF1(toList,predict, 'f')
# metrics.f1_score(df_train.gender.tail(182), predicted,average='weighted')
print("male",f1_male)
print("female",f1_female)
Average_f1=(f1_male + f1_female)/2
print("AVG F1",Average_f1)

male 0.8823529411764706
female 0.6666666666666667
AVG F1 0.7745098039215687


In [705]:
# finally using our pipline on the test set and hoping for good..
docs_test = df_test.story
predicted = text_clf_nb.predict(docs_test)

### Save output to csv
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [709]:
# converting the results into a dataframe
df_predicted = pd.DataFrame ({'test_example_id': df_test.index,'predicted_category': predicted})

In [710]:
df_predicted.to_csv('classification_results.csv',index=False)

In [576]:
# ####################################################
# 
# 
# 
# 
# 


In [577]:
gender=df_train.gender
story = df_train.story

In [464]:
# creating countVector with stop words max features and tokens on our train set

count_vect = CountVectorizer(stop_words=content_stop,max_features=50,min_df=2,tokenizer=my_tokenizer)

x=count_vect.fit_transform(story)
X_train_normalized = preprocessing.normalize(x, norm='l1')

encoder = LabelEncoder()
y = encoder.fit_transform(gender)


# split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X_train_normalized, y, test_size=0.2)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(291, 50)
(291,)
(73, 50)
(73,)


In [140]:
nb = MultinomialNB()
nb.fit(x_train, y_train)

print(nb.score(x_test, y_test))

0.7808219178082192


In [518]:

X_train_counts = count_vect.fit_transform(train_set.story)
X_train_counts.shape
my_matrix = X_train_counts.toarray()
# X_train_counts.vocabulary_.get("בן זוגי")


In [519]:
# using pipline
X_train_normalized = preprocessing.normalize(X_train_counts, norm='l1')
#X_train_normalized.toarray()

In [520]:
#  naive bayes classifier
clf = MultinomialNB().fit(X_train_normalized, train_set.story)
# count_vect.vocabulary_